In [ ]:
#FIND DATE WITH THE MOST TWEETS
dates = {}

for tweet in tw_coll.find():
	#check date of tweet
	messy_date = tweet["created_at"] #"Thu Mar 12 02:01:57 +0000 2020"
	real_date = messy_date[:10] + messy_date[-4:]

	if real_date in dates.keys():
		#increment counter for each date
		dates[real_date] += 1
	else:
		dates[real_date] = 1

best_date = None
for date in dates.keys():
	if best_date is None:
		best_date = date
	if dates[best_date] < dates[date]:
		best_date = date

print(best_date)
print(dates[best_date])

In [ ]:
#UNUSED CODE FOR CULLING RETURNED TWEETS ACCORDING TO USER CRITERIA

tweets_by_id = {}
final_tweets = {}
start_time = time.time()

try:
	for t in tweets:
		#we do a mapping of tweets to users this way so we can filter users based on ...location? eventually
		#user_tweet_mapping is an interim mapping we use to get the users from the database. At query time we add any additional criteria
		#tweets is a interm list of Tweets objects so we can keep the tweet id, users id, and connected_user id bundled during the batched query
		tweets_by_id[t.id] = t
		if t.user not in user_tweet_mapping:
			user_tweet_mapping[t.user] = []
		if t.connected_user not in user_tweet_mapping:
			user_tweet_mapping[t.connected_user] = []
		user_tweet_mapping[t.user].append(t.id) 
		user_tweet_mapping[t.connected_user].append(t.id)
		
		if len(user_tweet_mapping) > 4096:
			user_list = user_tweet_mapping.keys()
			users = tu_coll.find({"_id":{"$in":list(user_list)}})#list()}}) #get the users
			for user in users:
				# if the user is found by our constrained query, the Tweet object with the bundled tweet id, users id, and connected_user id
				# is added to the final_tweets list. The final_tweets list therefore only has tweets by users who meet our criteria
				for tweet_id in user_tweet_mapping[user['id']]:
					final_tweets[tweet_id] = tweets[tweet_id]
			user_tweet_mapping = {} #reset
except Exception as e:
	print(e)
finally:
	end_time = time.time()
	print("Tweets loaded: {}".format(len(final_tweets)))
	print("Time taken: {}".format(humanize.precisedelta(end_time - start_time, suppress=['days', 'milliseconds', 'microseconds'])))
	print("Memory used: {}".format(humanize.naturalsize(psutil.Process().memory_info().rss)))